In [7]:
import os

#if running on colab
colab = 'google.colab' in str(get_ipython())

if colab:
    content_path = "/content/drive/MyDrive/Core Images"
    project_path = "/content/"
else:
    project_path = os.getcwd()
    content_path = project_path + "/data/Core Images"
    
    
if colab:
    !rm "well-core-image-classification/" -r
    !git clone https://github.com/adhardy/well-core-image-classification.git



In [16]:
import importlib
import boto3

if colab:
  %cd $(project_path + "well-core-image-classification/")

import well_core_preprocessing as core_images
import well_core_AWS

#reload for debugging
importlib.reload(core_images)
importlib.reload(well_core_AWS)

if colab:
  %cd -

In [12]:
aws_creds_path = "awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = aws_creds_path
session = boto3.Session(profile_name='default')

bucket_name = "well-core"
well_core_s3 = well_core_AWS.WellCore_S3(session, bucket_name)

In [17]:
%cd $content_path

#clear directories
!rm slices/* cores/*

#get list of all images in directories
image_dirs = ["3-03- 11/","42-29- 6/"]


#list all the images in each directory
images = []
for dir in image_dirs:
    files = os.listdir(dir)
    dir_images = [os.path.join(dir, f) for f in files]
    images += dir_images

images = sorted(images)

core_x = 730
core_x_size = 6240
core_x_cm = 955
core_y_size = 865
core_x = (core_x,core_x + core_x_size)
core_y= (
    (1470,1470 + core_y_size),
    (2550,2550 + core_y_size),
    (3615,3615 + core_y_size)
)

slice_size = 224
step_size = slice_size/4

cores = core_images.CoreImages(images, core_x, core_y, core_x_cm, slice_size, step_size)

labels = "core_labels.csv"

cores.slice_cores(well_core_s3, "cores/", "slices/", labels, verbose = 1, save_local=True)

%cd -

/home/adam/well-core-image-classification/data/Core Images
rm: cannot remove 'slices/*': No such file or directory
rm: cannot remove 'cores/*': No such file or directory
Processing core photo: test image/S00066842.jpg
/home/adam/well-core-image-classification
